In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier


def drop_empty_items(data_frame: pd.DataFrame) -> pd.DataFrame:
    """Remove rows with zero values in 'Glucose', 'BloodPressure', or 'BMI'."""
    # Drop Insulin cause >30% missing values
    data_frame = data_frame.drop(['Insulin'], axis=1)
    return data_frame[(data_frame['Glucose'] != 0) & (data_frame['BloodPressure'] != 0) & (data_frame['BMI'] != 0)]


def scale_features(X_train, X_test, features):
    """Scale specified features using StandardScaler."""
    missing_features = [f for f in features if f not in X_train.columns or f not in X_test.columns]
    if missing_features:
        raise ValueError(f"Features not found in dataset: {missing_features}")

    scaler = StandardScaler()
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()

    X_train_scaled[features] = scaler.fit_transform(X_train[features])
    X_test_scaled[features] = scaler.transform(X_test[features])

    return X_train_scaled, X_test_scaled


# Load the PIMA dataset
df = pd.read_csv('data/diabetes.csv')

# Step 1: Remove rows with zero values in 'Glucose', 'BloodPressure', or 'BMI'
df = drop_empty_items(df)

# Define features
features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'BMI',
            'DiabetesPedigreeFunction', 'Age']

# Splitting dataset into 80-20 split
X = df.drop('Outcome', axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features after split (Z-scores, deviation from paper to avoid leakage)
X_train, X_test = scale_features(X_train, X_test, features)

# Apply SMOTE to training data only
# print("\nClass distribution before SMOTE:")
# print(y_train.value_counts(normalize=True))

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# print("\nClass distribution after SMOTE:")
# print(pd.Series(y_train_smote).value_counts(normalize=True))
# print(f"Train set shape after SMOTE: {X_train_smote.shape}")

# Define individual models and their hyperparameter grids (from Table 3)
# Random Forest
rf = RandomForestClassifier(random_state=42)
rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

# Gradient Boosting
gb = GradientBoostingClassifier(random_state=42)
gb_param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1]
}

# MLP
mlp = MLPClassifier(random_state=42, max_iter=2000, solver='adam')
mlp_param_grid = {
    'hidden_layer_sizes': [(50, 50)],
    'alpha': [0.001, 0.01],
    'learning_rate_init': [0.001, 0.01]
}

# XGBoost
xgb = XGBClassifier(random_state=42, eval_metric='logloss')
xgb_param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'colsample_bytree': [0.8, 1.0]
}

# Perform grid search for each model
models = [
    ('rf', rf, rf_param_grid),
    ('xgb', xgb, xgb_param_grid),
    ('mlp', mlp, mlp_param_grid),
    ('gb', gb, gb_param_grid)
]

best_estimators = {}
for name, model, param_grid in models:
    print(f"\nTuning {name}...")
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='f1_weighted',
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    grid_search.fit(X_train_smote, y_train_smote)
    best_estimators[name] = grid_search.best_estimator_
    print(f"Best {name} Hyperparameters: {grid_search.best_params_}")
    print(f"Best {name} F1-Weighted Score: {grid_search.best_score_:.4f}")

# Create ensemble model with soft voting
ensemble = VotingClassifier(
    estimators=[
        ('rf', best_estimators['rf']),
        ('xgb', best_estimators['xgb']),
        ('mlp', best_estimators['mlp']),
        ('gb', best_estimators['gb'])
    ],
    voting='soft',
    weights=[2, 1, 1, 1]
)

# Train ensemble on SMOTE-balanced training data
print("\nTraining ensemble model...")
ensemble.fit(X_train_smote, y_train_smote)

# Evaluate ensemble on test set
y_pred = ensemble.predict(X_test)

# Compute overall metrics
accuracy = accuracy_score(y_test, y_pred)
precision_overall, recall_overall, f1_overall, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

# Compute per-class metrics and specificity
precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred, average=None)
cm = confusion_matrix(y_test, y_pred)

# Compute specificity for each class
specificity = []
negative_counts = []
for i in range(2):  # Binary classification (0: Non-Diabetic, 1: Diabetic)
    tn = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])
    fp = cm[:, i].sum() - cm[i, i]
    specificity.append(tn / (tn + fp) if (tn + fp) > 0 else 0.0)
    negative_counts.append(tn + fp)

# Compute overall specificity (weighted by negative instances)
total_negative = sum(negative_counts)
specificity_overall = sum(
    spec * count for spec, count in zip(specificity, negative_counts)) / total_negative if total_negative > 0 else 0.0

# Print evaluation metrics
print("\nEnsemble Model Performance (Test Set):")
print("\nOverall Metrics:")
print(f"{'Metric':<15} {'Value':<10}")
print(f"{'Accuracy':<15} {accuracy:<10.4f}")
print(f"{'Precision':<15} {precision_overall:<10.4f}")
print(f"{'Recall':<15} {recall_overall:<10.4f}")
print(f"{'F1-Score':<15} {f1_overall:<10.4f}")
print(f"{'Specificity':<15} {specificity_overall:<10.4f}")

print("\nConfusion Matrix:")
print(f"{'':<15} {'Predicted Non-Diabetic':<22} {'Predicted Diabetic':<22}")
print(f"{'Actual Non-Diabetic':<15} {cm[0, 0]:<22} {cm[0, 1]:<22}")
print(f"{'Actual Diabetic':<15} {cm[1, 0]:<22} {cm[1, 1]:<22}")


Tuning rf...
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best rf Hyperparameters: {'bootstrap': False, 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Best rf F1-Weighted Score: 0.8094

Tuning xgb...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best xgb Hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Best xgb F1-Weighted Score: 0.7996

Tuning mlp...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best mlp Hyperparameters: {'alpha': 0.01, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.001}
Best mlp F1-Weighted Score: 0.8204

Tuning gb...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best gb Hyperparameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}
Best gb F1-Weighted Score: 0.7978

Training ensemble model...

Ensemble Model Performance (Test Set):

Overall Metrics:
Metric          Value     